https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html#notes

In [6]:
import gymnasium as gym
import os
import csv

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env


rollout/
ep_len_mean: Mean episode length (averaged over stats_window_size episodes, 100 by default)

ep_rew_mean: Mean episodic training reward (averaged over stats_window_size episodes, 100 by default), a Monitor wrapper is required to compute that value (automatically added by make_vec_env).

exploration_rate: Current value of the exploration rate when using DQN, it corresponds to the fraction of actions taken randomly (epsilon of the “epsilon-greedy” exploration)

success_rate: Mean success rate during training (averaged over stats_window_size episodes, 100 by default), you must pass an extra argument to the Monitor wrapper to log that value (info_keywords=("is_success",)) and provide info["is_success"]=True/False on the final step of the episode

time/
episodes: Total number of episodes

fps: Number of frames per seconds (includes time taken by gradient update)

iterations: Number of iterations (data collection + policy update for A2C/PPO)

time_elapsed: Time in seconds since the beginning of training

total_timesteps: Total number of timesteps (steps in the environments)

train/
actor_loss: Current value for the actor loss for off-policy algorithms

approx_kl: approximate mean KL divergence between old and new policy (for PPO), it is an estimation of how much changes happened in the update

clip_fraction: mean fraction of surrogate loss that was clipped (above clip_range threshold) for PPO.

clip_range: Current value of the clipping factor for the surrogate loss of PPO

critic_loss: Current value for the critic function loss for off-policy algorithms, usually error between value function output and TD(0), temporal difference estimate

ent_coef: Current value of the entropy coefficient (when using SAC)

ent_coef_loss: Current value of the entropy coefficient loss (when using SAC)

entropy_loss: Mean value of the entropy loss (negative of the average policy entropy)

explained_variance: Fraction of the return variance explained by the value function, see https://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score (ev=0 => might as well have predicted zero, ev=1 => perfect prediction, ev<0 => worse than just predicting zero)

learning_rate: Current learning rate value

loss: Current total loss value

n_updates: Number of gradient updates applied so far

policy_gradient_loss: Current value of the policy gradient loss (its value does not have much meaning)

value_loss: Current value for the value function loss for on-policy algorithms, usually error between value function output and Monte-Carlo estimate (or TD(lambda) estimate)

std: Current standard deviation of the noise when using generalized State-Dependent Exploration (gSDE)

In [7]:
# Parallel environments
vec_env = make_vec_env("CartPole-v1", n_envs=4)

#Single vectorized enviroment
env = make_vec_env("CartPole-v1")

#create csv file for logging the callbacks
model = PPO("MlpPolicy", env, verbose=1)    #verbose 0-> no output, verbose 1-> info messages, verbose 2-> debug messages

        
model.learn(total_timesteps=100000) #max 200 steps/episode -> 100.000 steps = 5000 episodes, 50 
model.save("ppo_cart_pole_2")

# for discrete car:
#reached minimum ep_rew_mean after 254 iterations/ 2530 n_updates
#reached first time a reward > -200 after 171 iterations/ 1700 n_updates

Using cpu device


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.1     |
|    ep_rew_mean     | 20.1     |
| time/              |          |
|    fps             | 500      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 23.5        |
|    ep_rew_mean          | 23.5        |
| time/                   |             |
|    fps                  | 362         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008162603 |
|    clip_fraction        | 0.0888      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.0228     |
|    learning_rate        | 0.

In [8]:
del model # remove to demonstrate saving and loading

model = PPO.load("ppo_cart_pole_2")

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render("human")

: 

In [ ]:
model = PPO.load("ppo_mountain_cart_v0")

print(model.rollout_buffer)